In [1]:
import os
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# Number of Historical Days used as Input!
num_days_history = 7

In [3]:
class CustomDataset(Dataset):
    def __init__(self, start_pt, num_pts):
        self.start_pt = start_pt
        self.num_pts = num_pts

        base = "data/"
        self.input_folder = base + "input/"
        self.output_folder = base + "output/"
        self.input_files = os.listdir(self.input_folder)
        self.output_files = os.listdir(self.output_folder)

        assert (start_pt + num_pts) < len(self.output_files)

    def __len__(self):
        return self.num_pts

    def __getitem__(self, idx):
        input = []
        for i in range(num_days_history):
            input.append(torch.load(self.input_folder + self.input_files[self.start_pt + idx + 67 - i]))
        input = torch.cat(input, 0)
        output = torch.load(self.output_folder + self.output_files[self.start_pt + idx])
        return input, output.squeeze().long()

In [4]:
BATCH_SIZE = 1
train_dataloader = DataLoader(CustomDataset(0, 4680), batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(CustomDataset(4680, 730), batch_size=BATCH_SIZE, shuffle=False)

In [5]:
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

Feature batch shape: torch.Size([1, 42, 676, 407])
Labels batch shape: torch.Size([1, 676, 407])


Model

In [6]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        convs = []
        conv_chs = [6 * num_days_history, 512, 256, 64, 16, 2]
        for i in range(len(conv_chs) - 1):
            convs.append(nn.Conv2d(conv_chs[i], conv_chs[i + 1], kernel_size=(5,5), padding="same"))
        self.convs = nn.ModuleList(convs)
        self.act = nn.ReLU()
        self.LogSoftmax = nn.LogSoftmax(dim=1)
        # self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        for conv in self.convs:
            x = self.act(conv(x))
        x = self.LogSoftmax(x)
        return x

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


In [8]:
BATCH_SIZE = 1
# train_dataloader = DataLoader(CustomDataset(0, 4680), batch_size=BATCH_SIZE, shuffle=True)
# test_dataloader = DataLoader(CustomDataset(4680, 730), batch_size=BATCH_SIZE, shuffle=False)

train_dataloader = DataLoader(CustomDataset(0, 8), batch_size=BATCH_SIZE)

train_steps = len(train_dataloader.dataset) // BATCH_SIZE

In [9]:
# weight = 5
# def weightedBCELoss(output, target):
#     loss = (weight * (target * torch.log(output))) + ((1 - target) * torch.log(1 - output))
#     return torch.neg(torch.mean(loss))

model = Net().to(device)
loss_fn = nn.NLLLoss(torch.Tensor([1, 50]).to(device))
optimizer = optim.Adam(model.parameters(), lr=0.001)

print(model)

Net(
  (convs): ModuleList(
    (0): Conv2d(42, 512, kernel_size=(5, 5), stride=(1, 1), padding=same)
    (1): Conv2d(512, 256, kernel_size=(5, 5), stride=(1, 1), padding=same)
    (2): Conv2d(256, 64, kernel_size=(5, 5), stride=(1, 1), padding=same)
    (3): Conv2d(64, 16, kernel_size=(5, 5), stride=(1, 1), padding=same)
    (4): Conv2d(16, 2, kernel_size=(5, 5), stride=(1, 1), padding=same)
  )
  (act): ReLU()
  (LogSoftmax): LogSoftmax(dim=1)
)


In [10]:
best_model_value = 0

n_epochs = 5
for epoch in range(n_epochs):
    model.train()

    total_train_loss = 0
    train_correct = 0
    train_recall = 0
    train_precision = 0
    for inputs, labels in tqdm(train_dataloader):
        inputs, labels = inputs.to(device), labels.to(device)
        
        y_pred = model(inputs)
        loss = loss_fn(y_pred, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_loss += loss

        # pred = y_pred > 0.5
        pred = torch.argmax(y_pred, dim=1)
        
        train_correct += ((pred == labels).float().sum().item() / labels.numel())
        train_recall += ((pred[labels > 0] == labels[labels > 0]).float().sum().item() / labels[labels > 0].numel())
        if labels[pred > 0].numel() > 0:
            train_precision += ((pred[pred > 0] == labels[pred > 0]).float().sum().item() / labels[pred > 0].numel())
    
    avg_train_loss = total_train_loss / train_steps
    train_correct = train_correct / train_steps
    train_recall = train_recall / train_steps
    train_precision = train_precision / train_steps
    
    print("Epoch: %d, Loss: %.4f, Accuracy: %.4f, Recall: %.4f, Precision: %.4f" % (epoch+1, avg_train_loss, 100*train_correct, 100*train_recall, 100*train_precision))

    model_value = 0
    if (train_recall + train_precision) > 1e-4:
        model_value = (train_recall * train_precision) / (train_recall + train_precision)
    print("Model Value:", model_value)
    if model_value > best_model_value:
        print("Saving Model!")
        torch.save(model.state_dict(), "model.pt")
        best_model_value = model_value


100%|██████████| 8/8 [01:09<00:00,  8.68s/it]


Epoch: 1, Loss: 0.7089, Accuracy: 98.4601, Recall: 0.0000, Precision: 0.0000
Model Value: 0


100%|██████████| 8/8 [01:01<00:00,  7.71s/it]


Epoch: 2, Loss: 0.6926, Accuracy: 98.5096, Recall: 0.0000, Precision: 0.0000
Model Value: 0


100%|██████████| 8/8 [01:01<00:00,  7.65s/it]


Epoch: 3, Loss: 0.6931, Accuracy: 98.5096, Recall: 0.0000, Precision: 0.0000
Model Value: 0


100%|██████████| 8/8 [01:01<00:00,  7.65s/it]


Epoch: 4, Loss: 0.6931, Accuracy: 98.5096, Recall: 0.0000, Precision: 0.0000
Model Value: 0


100%|██████████| 8/8 [01:01<00:00,  7.67s/it]

Epoch: 5, Loss: 0.6931, Accuracy: 98.5096, Recall: 0.0000, Precision: 0.0000
Model Value: 0
